# Data I/O

In [1]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import sys

Let us start with a small file in CSV format (comma-separated values):

In [2]:
!cat Files-06/example-1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,bye


We can use read_csv() to obtain a DataFrame:

In [3]:
df = pd.read_csv( 'Files-06/example-1.csv' )
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,bye


A file will not always include a header row (column names), as in the following example:

In [4]:
!cat Files-06/example-2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,bye


In this case we can manually specify column names, or let Pandas generate them automatically:

In [5]:
pd.read_csv( 'Files-06/example-2.csv', header=None )

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,bye


In [6]:
pd.read_csv( 'Files-06/example-2.csv', names=['a', 'b', 'c', 'd', 'message'] )

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,bye


If we wish $message$ to be the index of the DataFrame, we can specify it through the $index\_col$ parameter:

In [7]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv( 'Files-06/example-2.csv', names=names, index_col='message' )

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
bye,9,10,11,12


We can build a hierarchical index providing a list of indices:

In [8]:
!cat Files-06/example-3.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [9]:
parsed = pd.read_csv( 'Files-06/example-3.csv', index_col=[ 'key1', 'key2'] )
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In other cases the file could not use a fixed delimiter, but whitespaces or some other pattern to separate fields. In these cases we can use a regular expression as _sep_:

In [10]:
!cat Files-06/example-4.txt

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


In [11]:
result = pd.read_csv( 'Files-06/example-4.txt', sep='\s+' ) # \s+ is a regular expression indicating 
                                                       # "one or more whitespace characters"
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


Given that the file contained a header column less than data columns, Pandas has inferred that the first column in the file was the index.

The $skiprows$ parameter allows to ignore certain rows in the file:

In [12]:
!cat Files-06/example-5.csv

# hey!
a,b,c,d,message
# I just wanted to spice things up a bit
# Who reads CSV files using a text editor?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,bye


In [13]:
pd.read_csv( 'Files-06/example-5.csv', skiprows=[0, 2, 3] )

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,bye


When rows to ignore have a clear format (e.g., comments starting with $#$, we can use a regular expression for the $comment$ parameter, which automatically recognizes comment rows:

In [14]:
pd.read_csv( 'Files-06/example-5.csv', comment="#" )

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,bye


Managing unknown values is an integral part of the data input process. Generally speaking, missing data are either not present (empty strings) or marked using a sentinel value. Pandas recognizes some common placeholders by default, such as $NA$, $-1.#INF$, or $NULL$:

In [15]:
!cat Files-06/example-6.csv

thing,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,bye


In [16]:
result = pd.read_csv( 'Files-06/example-6.csv' )
result

,thing,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,bye


In [17]:
result.isnull()

,thing,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


The $na\_values$ parameter receives a list or set of strings to consider as missing values, which are added to those recognized by default:

In [18]:
result = pd.read_csv( 'Files-06/example-6.csv', na_values=['bye'] )
result

,thing,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


If we do not wish to preserve the default definitions, we can overwrite them:

In [19]:
pd.read_csv( 'Files-06/example-6.csv', na_values=['bye'], keep_default_na=False )

,thing,a,b,c,d,message
0,one,1,2,3,4,NA
1,two,5,6,,8,world
2,three,9,10,11,12,NaN


We can specify different sentinels for different columns using a dictionary:

In [20]:
pd.read_csv( 'Files-06/example-6.csv', na_values={'message': ['bye'], 'thing': ['two']}, keep_default_na=False )

,thing,a,b,c,d,message
0,one,1,2,3,4,NA
1,NaN,5,6,,8,world
2,three,9,10,11,12,NaN


Nótese cómo en la columna $\texttt{c}$ se ha preservado la definición por defecto de valores NA, dado que no se ha incluido entre las definiciones específicas en $\texttt{na_values}$.

### Reading files in chunks

When processing large files, and due to memory considerations, we may wish to read them in a chunked fashion:

In [21]:
result = pd.read_csv( 'Files-06/example-7.csv' )
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
5,1.817480,0.742273,0.419395,-2.251035,Q
6,-0.776764,0.935518,-0.332872,-1.875641,U
7,-0.913135,1.530624,-0.572657,0.477252,K
8,0.358480,-0.497572,-0.367016,0.507702,S
9,-1.740877,-1.160417,-1.637830,2.172201,G


We can read only a few rows at the start of the file:

In [22]:
pd.read_csv( 'Files-06/example-7.csv', nrows=5 )

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


To read a file in a chunked way, we specify $chunksize$ as a number of rows:

In [23]:
chunker = pd.read_csv( 'Files-06/example-7.csv', chunksize=1000 )
chunker

The chunker object allows to iterate over the rows of the file 1000 at a time, processing them (e.g., aggregating the value counts for the $key$ column:

In [24]:
tot = Series([])
for piece in chunker:
    tot = tot.add( piece['key'].value_counts(), fill_value=0 )

tot = tot.sort_values( ascending=False )
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

The TextFileReader class comes equipped with a get_chunk() method, which allows to traverse the file in chunks of arbitrary sizes (not necessarily that passed to read_csv()).

### Writing data in text format

We can also export Pandas objects to CSV:

In [25]:
data = pd.read_csv( 'Files-06/example-6.csv' )
data

,thing,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,bye


In [26]:
data.to_csv( '/tmp/output.csv' )
!cat /tmp/output.csv

,thing,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,bye


We could change the delimiter:

In [27]:
data.to_csv( sys.stdout, sep='|' ) # We write to sys.stdout, so that the output comes through the screen

|thing|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|bye


Missing values appear as empty strings in the output. We can change them for a different sentinel value:

In [28]:
data.to_csv( sys.stdout, na_rep='NULL' )

,thing,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,bye


We can disable writing the row/column labels:

In [29]:
data.to_csv( sys.stdout, index=False, header=False )

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,bye


We can write a subset of columns only, and in a particular order:

In [30]:
data.to_csv( sys.stdout, index=False, columns=['a', 'c', 'b'] )

a,c,b
1,3.0,2
5,,6
9,11.0,10


Series objects also have a to_csv() method:

In [31]:
dates = pd.date_range( '1/1/2010', periods=7 )
dates

DatetimeIndex(['2010-01-01', '2010-01-02', '2010-01-03', '2010-01-04',
               '2010-01-05', '2010-01-06', '2010-01-07'],
              dtype='datetime64[ns]', freq='D')

In [32]:
ts = Series( np.arange(7), index=dates )
ts

2010-01-01    0
2010-01-02    1
2010-01-03    2
2010-01-04    3
2010-01-05    4
2010-01-06    5
2010-01-07    6
Freq: D, dtype: int64

In [33]:
ts.to_csv( sys.stdout )

2010-01-01,0
2010-01-02,1
2010-01-03,2
2010-01-04,3
2010-01-05,4
2010-01-06,5
2010-01-07,6


## Binary formats

One of the more convenient ways to store and read Pandas objects is as pickle objects, through to_pickle() and from_pickle():

In [34]:
frame = pd.read_csv( 'Files-06/example-1.csv' )
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,bye


In [35]:
frame.to_pickle( '/tmp/object.pickle' )

In [36]:
pd.read_pickle( '/tmp/object.pickle' )

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,bye


### Reading Excel files

In [37]:
df = pd.read_excel( 'Files-06/example-8.xlsx' )
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,NaN


## Interacting with SQL databases

Let us use the $sqlite3$ module to build a simple in-memory database with which to interact:

In [38]:
import sqlite3
import pandas.io.sql as sql

In [39]:
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d REAL
);"""
con = sqlite3.connect( ':memory:' )
con.execute( query )
con.commit()

Insertamos valores en la BBDD:

In [40]:
data = [('Paris', 'France',  48.51, 2.21),
        ('Madrid', 'Spain',  40.23, -3.43),
        ('Berlin', 'Germany', 52.31, 13.23)]
stmt = 'INSERT INTO test VALUES (?, ?, ?, ?)'

con.executemany( stmt, data )
con.commit()

Most SQL drivers return a list of tuples when we SELECT from a table:

In [41]:
cursor = con.execute( 'select * from test' )
rows = cursor.fetchall()
rows

[('Paris', 'France', 48.51, 2.21),
 ('Madrid', 'Spain', 40.23, -3.43),
 ('Berlin', 'Germany', 52.31, 13.23)]

We can feed this list of tuples to the DataFrame() constructor. We also need the schema for the columns, contained in the $description$ attribute of the cursor:

In [42]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [43]:
DataFrame( rows, columns=next(zip(*cursor.description)) )

,a,b,c,d
0,Paris,France,48.51,2.21
1,Madrid,Spain,40.23,-3.43
2,Berlin,Germany,52.31,13.23


But it would be much more convenient no the perform these operations to read a full table, which is actually what we are doing. The pandas.io.sql module simplifies the creation of the DataFrame:

In [44]:
sql.read_sql( 'select * from test', con )

,a,b,c,d
0,Paris,France,48.51,2.21
1,Madrid,Spain,40.23,-3.43
2,Berlin,Germany,52.31,13.23


### Creating and reading tables using Pandas value objects

New Pandas versions are developing an interface to completely integrate Pandas objects with SQL. This functionality is implemented on top of SQLAlchemy, a Python module to interact with SQL databases in a driver-agnostic fashion:

In [45]:
import sqlalchemy as sa
import pandas as pd

Let us create a SQL table inside a sqlite3 file containing the information about Tesla Motors stock obtained through Yahoo! Finance:

In [46]:
import pandas_datareader.data as web
tsla = web.get_data_yahoo("TSLA")
tsla.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-06-29,25.00,17.540001,19.000000,23.889999,18766300,23.889999
2010-06-30,30.42,23.299999,25.790001,23.830000,17187100,23.830000
2010-07-01,25.92,20.270000,25.000000,21.959999,8218800,21.959999
2010-07-02,23.10,18.709999,23.000000,19.200001,5139800,19.200001
2010-07-06,20.00,15.830000,20.000000,16.110001,6866900,16.110001


We build the databse connection engine:

In [47]:
engine = sa.create_engine( "sqlite:////tmp/db.sqlite" )

And we create a SQLDatabase object to abstract this connection:

In [48]:
db = pd.io.sql.SQLDatabase( engine )

Pandas includes a SQLTable class that associates a DataFrame to an SQL table. This class includes functions to automatically create the SQL schema from the columns of the DataFrame and their data types:

In [49]:
tb = pd.io.sql.SQLTable( name="tsla", pandas_sql_engine=db, frame=tsla )

As we can see, the object connects a table called "tsla" in the database with our DataFrame. To create the table:

In [50]:
tb.create()

In [51]:
!sqlite3 -batch /tmp/db.sqlite <<< ".schema"

CREATE TABLE tsla (
	"Date" DATETIME, 
	"High" FLOAT, 
	"Low" FLOAT, 
	"Open" FLOAT, 
	"Close" FLOAT, 
	"Volume" BIGINT, 
	"Adj Close" FLOAT
);
CREATE INDEX "ix_tsla_Date" ON tsla ("Date");


In [52]:
!sqlite3 -batch /tmp/db.sqlite <<< "select * from tsla;"

The hable has been created inside the DB, but no rows have been inserted. To do this:

In [53]:
tb.insert()

In [54]:
!sqlite3 -batch /tmp/db.sqlite <<< "select * from tsla;"

2010-06-29 00:00:00.000000|25.0|17.5400009155273|19.0|23.8899993896484|18766300|23.8899993896484
2010-06-30 00:00:00.000000|30.4200000762939|23.2999992370605|25.7900009155273|23.8299999237061|17187100|23.8299999237061
2010-07-01 00:00:00.000000|25.9200000762939|20.2700004577637|25.0|21.9599990844727|8218800|21.9599990844727
2010-07-02 00:00:00.000000|23.1000003814697|18.7099990844727|23.0|19.2000007629395|5139800|19.2000007629395
2010-07-06 00:00:00.000000|20.0|15.8299999237061|20.0|16.1100006103516|6866900|16.1100006103516
2010-07-07 00:00:00.000000|16.6299991607666|14.9799995422363|16.3999996185303|15.8000001907349|6921700|15.8000001907349
2010-07-08 00:00:00.000000|17.5200004577637|15.5699996948242|16.1399993896484|17.4599990844727|7711400|17.4599990844727
2010-07-09 00:00:00.000000|17.8999996185303|16.5499992370605|17.5799999237061|17.3999996185303|4050600|17.3999996185303
2010-07-12 00:00:00.000000|18.0699996948242|17.0|17.9500007629395|17.0499992370605|2202500|17.0499992370605
20

To read this table, we reset the kernel and start from a new engine object.

In [55]:
import sqlalchemy as sa
import pandas as pd

In [56]:
engine=sa.create_engine("sqlite:////tmp/db.sqlite")

Next, we should create an SQLDatabase object using this engine. However, due to a Pandas naïvety, DB metadata are not correctly initialized so we need to perform one additional step using SQLAlchemy:

In [57]:
meta = sa.MetaData()
tb = sa.Table( "tsla", meta, autoload_with=engine )

This call reads the metadata associated to the table. Now we can create the SQLDatabase object:

In [58]:
db = pd.io.sql.SQLDatabase( engine, meta=meta )

And read the table using this object:

In [59]:
df = db.read_table("tsla")
df.head()

,Date,High,Low,Open,Close,Volume,Adj Close
0,2010-06-29,25.00,17.540001,19.000000,23.889999,18766300,23.889999
1,2010-06-30,30.42,23.299999,25.790001,23.830000,17187100,23.830000
2,2010-07-01,25.92,20.270000,25.000000,21.959999,8218800,21.959999
3,2010-07-02,23.10,18.709999,23.000000,19.200001,5139800,19.200001
4,2010-07-06,20.00,15.830000,20.000000,16.110001,6866900,16.110001


Note that the index has not been correctly recovered. We could pivot it with set_index(). Another option would be to indicate which column to use as index upon reading the table:

In [60]:
df = db.read_table( "tsla", index_col="Date" )
df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-06-29,25.00,17.540001,19.000000,23.889999,18766300,23.889999
2010-06-30,30.42,23.299999,25.790001,23.830000,17187100,23.830000
2010-07-01,25.92,20.270000,25.000000,21.959999,8218800,21.959999
2010-07-02,23.10,18.709999,23.000000,19.200001,5139800,19.200001
2010-07-06,20.00,15.830000,20.000000,16.110001,6866900,16.110001
